In [33]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.python.keras.engine.base_preprocessing_layer import PreprocessingLayer
import requests
import urllib.parse
import math
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import cupy as cp
import seaborn
from numba import cuda
import dataset
from importlib import reload
import os.path

reload(dataset)
print(tf.config.list_physical_devices('GPU'))
tf.compat.v1.enable_eager_execution()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras
keras.backend.set_epsilon(1)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2070 Super, pci bus id: 0000:01:00.0, compute capability: 7.5



In [43]:
def savedata(xtrain, ytrain, xtest, ytest, name="data"):
    np.savez_compressed(os.path.join("saveddata",name), timetrain=xtrain[0], idxtrain=xtrain[1], typetrain=xtrain[2], totidxtrain=xtrain[3], ytrain=ytrain,
                       timetest=xtest[0], idxtest=xtest[1], typetest=xtest[2], totidxtest=xtest[3], ytest=ytest)
def loaddata(name="data"):
    with np.load(os.path.join("saveddata",(name if name.endswith(".npz") else name+".npz"))) as data:
        xtrain = [data["timetrain"], data["idxtrain"], data["typetrain"], data["totidxtrain"]]
        ytrain = data["ytrain"]
        xtest = [data["timetest"], data["idxtest"], data["typetest"], data["totidxtest"]]
        ytest = data["ytest"]
    return xtrain, ytrain, xtest, ytest

In [57]:
lookback=60
xtrain, ytrain, xtest, ytest = dataset.getdata(lookback=lookback, items = [
    ("★ M9 Bayonet | Fade (Factory New)", "Knife"),
    ("StatTrak™ AK-47 | Ice Coaled (Factory New)", "Gun"),
    ("Recoil Case", "Case"),
    ("CS:GO Weapon Case", "Case"),
    ("Sticker | s1mple (Gold) | Katowice 2019", "Sticker"),
    ("Sealed Graffiti | NaCl (Shark White)", "Graffiti"),
    ("Canals Pin", "Pin"),
    ("StatTrak™ Music Kit | Hundredth, FREE", "Music"),
    ("Stockholm 2021 Challengers Patch Pack", "Patch"),
    ("Number K | The Professionals", "Agent"),
    ("★ Broken Fang Gloves | Needle Point (Field-Tested)", "Gloves"),
    ("AWP | Asiimov (Well-Worn)", "Gun"),
    # ("AWP | Asiimov (Factory New)", "Gun"),
    # ("★ Broken Fang Gloves | Needle Point (Well-Worn)", "Gloves"),
])
savedata(xtrain, ytrain, xtest, ytest)

★ M9 Bayonet | Fade (Factory New)
StatTrak™ AK-47 | Ice Coaled (Factory New)
Recoil Case
CS:GO Weapon Case
Sticker | s1mple (Gold) | Katowice 2019
Sealed Graffiti | NaCl (Shark White)
Canals Pin
StatTrak™ Music Kit | Hundredth, FREE
Stockholm 2021 Challengers Patch Pack
Number K | The Professionals
★ Broken Fang Gloves | Needle Point (Field-Tested)
AWP | Asiimov (Well-Worn)
timeinputstrain.shape=(14816, 60, 1)
idxinputstrain.shape=(14816,)
typeinputstrain.shape=(14816,)
totidxinputstrain.shape=(14816,)
ytrain.shape=(14816, 1)
timeinputstest.shape=(3704, 60, 1)
idxinputstest.shape=(3704,)
typeinputstest.shape=(3704,)
totidxinputstest.shape=(3704,)
ytest.shape=(3704, 1)


In [58]:
%%time
xtrain, ytrain, xtest, ytest = loaddata()

CPU times: total: 31.2 ms
Wall time: 46.9 ms


In [82]:
rnnlayer = layers.GRU
def getmodel():
    """m = keras.Sequential([
        ,
        layers.Dense(80),
        layers.GRU(80, return_sequences=False, input_shape=(lookback, 1)),
        layers.Dense(25),
        layers.Dense(1),
    ])"""
    
    vocab = ["Gloves", "Agent", "Patch", "Music", "Pin", "Graffiti", "Sticker", "Case", "Knife", "Gun"]
    timeinput = keras.Input(shape=(lookback, 1))
    idxinput = keras.Input(shape=(1), dtype=tf.float32)
    typeinput = keras.Input(shape=(1), dtype=tf.string)
    totalidxinput = keras.Input(shape=(1), dtype=tf.float32)
    
    typeinput2 = layers.StringLookup(vocabulary=vocab, output_mode="multi_hot")(typeinput)
    typeinput2 = layers.Reshape((typeinput2.shape[1], 1))(typeinput2)
    
    idxinput2 = layers.Reshape((idxinput.shape[1],1))(idxinput)
    totalidxinput2 = layers.Reshape((idxinput.shape[1],1))(totalidxinput)
    
    
    x = layers.Concatenate(axis=1)([timeinput, idxinput2, typeinput2, totalidxinput2])
    x = layers.Reshape((1, x.shape[1]))(x)
    
    x = layers.Dense(200)(x)
    x = rnnlayer(200, return_sequences=True)(x)
    x = layers.Dense(120)(x)
    x = rnnlayer(120, return_sequences=True)(x)
    x = layers.Dense(80)(x)
    x = rnnlayer(80, return_sequences=True)(x)
    x = layers.Dense(40)(x)
    x = rnnlayer(40, return_sequences=False)(x)
    # x = layers.Dense(25)(x)
    for i in range(5):
        x = layers.Dense(25)(x)
    x = layers.Dense(1)(x)
    x = layers.Reshape((1,))(x)
    x = layers.ReLU()(x)
    
    m = keras.Model(inputs=[timeinput, idxinput, typeinput, totalidxinput], outputs=x)
    
    m.build((None, lookback, 1))
    m.summary()
    return m
model=getmodel()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_39 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_38 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 string_lookup_9 (StringLookup)  (None, 11)          0           ['input_39[0][0]']               
                                                                                                  
 input_40 (InputLayer)          [(None, 1)]          0           []                               
                                                                                            

In [ ]:
%%time
model.compile(optimizer="adam",loss="mean_absolute_percentage_error", metrics=["accuracy", keras.metrics.RootMeanSquaredError(), keras.metrics.MeanAbsolutePercentageError()])
history=model.fit(xtrain, ytrain, batch_size=32, epochs=100, verbose=1)

Epoch 1/100
463/463 [==============================] - 10s 13ms/step - loss: 87.0355 - accuracy: 2.0248e-04 - root_mean_squared_error: 249.1617 - mean_absolute_percentage_error: 87.0355
Epoch 2/100
463/463 [==============================] - 6s 12ms/step - loss: 86.8430 - accuracy: 1.3499e-04 - root_mean_squared_error: 249.1412 - mean_absolute_percentage_error: 86.8430
Epoch 3/100
463/463 [==============================] - 6s 12ms/step - loss: 87.2619 - accuracy: 1.3499e-04 - root_mean_squared_error: 249.0327 - mean_absolute_percentage_error: 87.2619
Epoch 4/100
463/463 [==============================] - 6s 12ms/step - loss: 86.8983 - accuracy: 6.7495e-05 - root_mean_squared_error: 249.1751 - mean_absolute_percentage_error: 86.8983
Epoch 5/100
463/463 [==============================] - 6s 12ms/step - loss: 86.7989 - accuracy: 2.6998e-04 - root_mean_squared_error: 249.0764 - mean_absolute_percentage_error: 86.7989
Epoch 6/100
463/463 [==============================] - 5s 12ms/step - loss

In [71]:
model.save("model.hdf5")

In [72]:
model = models.load_model("model.hdf5")

In [51]:
pred = model.predict(xtrain)
print(pred.shape)
print(ytrain.shape)
print(type(pred))
# pred = scaler.inverse_transform(pred)

print("Training MAPE",np.mean(np.abs((ytrain - pred) / ytrain)) * 100)
print("Average testing distance percentage:", np.mean(np.divide(np.absolute(pred-ytrain),ytrain))*100)
print(ytrain)
print(pred)

463/463 [==============================] - 3s 5ms/step
(14816, 1)
(14816, 1)
<class 'numpy.ndarray'>
Training MAPE 321.45890356900236
Average testing distance percentage: 321.45890356900236
[[5.5960e+01]
 [8.7000e-01]
 [3.0000e-02]
 ...
 [1.3300e+00]
 [2.3190e+01]
 [4.7442e+02]]
[[ 22.636389 ]
 [  2.2465422]
 [  2.2465422]
 ...
 [  2.2465422]
 [  8.147372 ]
 [142.52293  ]]


In [52]:
pred = model.predict(xtest)
print("Testing MAPE",np.mean(np.abs((ytest - pred) / ytest)) * 100)
print("Average testing distance percentage:", np.mean(np.divide(np.absolute(pred-ytest),ytest))*100)
print(ytest)
print(pred)

116/116 [==============================] - 2s 7ms/step
Testing MAPE 353.48439675358094
Average testing distance percentage: 353.48439675358094
[[219.84]
 [  0.82]
 [  2.62]
 ...
 [  2.46]
 [  3.47]
 [ 81.66]]
[[142.52293  ]
 [  2.2465422]
 [  2.2465422]
 ...
 [  2.4940498]
 [  2.2465422]
 [ 55.335846 ]]


In [66]:
model.evaluate(xtrain, ytrain)

463/463 [==============================] - 5s 8ms/step - loss: 64.9942 - accuracy: 3.3747e-04 - root_mean_squared_error: 190.4695 - mean_absolute_percentage_error: 64.9942


[64.99420166015625,
 0.00033747300039976835,
 190.46946716308594,
 64.99420166015625]

In [67]:
model.evaluate(xtest, ytest)

116/116 [==============================] - 2s 10ms/step - loss: 64.6543 - accuracy: 5.3996e-04 - root_mean_squared_error: 195.1941 - mean_absolute_percentage_error: 64.6543


[64.6543197631836, 0.0005399567889980972, 195.19407653808594, 64.6543197631836]

In [55]:
model.predict([np.zeros((1,60, 1)), np.asarray([42069]), np.asarray(["Knife"]), np.asarray([100100])])

1/1 [==============================] - 0s 32ms/step


array([[0.]], dtype=float32)

In [13]:
tf.keras.backend.clear_session()

In [14]:
device = cuda.get_current_device()
device.reset()